In [29]:
import pandas as pd
from tqdm import  tqdm

<h1>Préparation des données partie 2</h1>
Dans cette partie nous allons faire la construction du vocabulaire et extraction des caractéristiques

In [30]:
data=pd.read_csv('./emails.csv')
X=data['text']
X.head()

0    emailaddr mon aug number numbernumbernumb numb...
1    emailaddr wed sep number numbernumbernumb numb...
2    emailaddr sat oct number numbernumbernumb numb...
3    senttonumbernumbernumberzzzzemailaddr fri sep ...
4    emailaddr wed oct number numbernumbernumb numb...
Name: text, dtype: object

<h2>1-Construction du Vocabulaire<h2>
<h3>1.1 Géneration du vocabulaire<h3>


In [31]:
def update_word_list(email:str,word_list:dict):
    words = email.split(' ')
    
    for word in words:
        if word not in word_list:
            word_list[word] = 1
        else:
            word_list[word] += 1

In [32]:
word_list=dict() #dictionnaire du vocabulaire sous la forme {mot : nombre d'apparition}
for i in tqdm(range(len(X))):
    update_word_list(X[i],word_list)

100%|██████████| 9349/9349 [00:00<00:00, 9493.22it/s] 


In [33]:
def build_vocabulary(word_list:dict,k:int)->list:
    vocabulary = [word for word, count in word_list.items() if count >=k ]
    return  vocabulary

In [34]:
#Génération du vocabulaire dans un fichier
k=3
vocabulary=build_vocabulary(word_list, k)
with open('vocab.txt','w') as f:
    for i in vocabulary:
        f.write(f"{i}\n")
len(vocabulary)

31550

<h3>1.2 Indéxation des mots de chaque mail</h3>

In [35]:
def get_vocabulary(filename:str)->list:
    with open(filename, 'r') as file:
        # lire le fichier en tant que texte
        text = file.read()
        # convertir le texte en liste de mots
        vocabulary_list = text.split()
    return vocabulary_list


In [36]:
def indexation(vocabList:list,email:str)->list:
    index=list()
    #Découper l'email en tableau de mot
    words = email.split(' ')
    #Parcourir chaque mot dans l'email et trouver son index dans la liste de vocabulaire
    for word in words:
        if word in vocabList:
            #Ajouter l'index du mot dans la liste d'index de mots
            index.append(vocabList.index(word))
    return index

In [37]:
def generateAllIndexes(vocabList:list,emails)->list:
    list_index=list()
    for i in tqdm(range(len(emails))):
        list_index.append(indexation(vocabList,X[i]))
    return list_index

<h2>Extraction des caractéristiques</h2>
Représentation des caractéristiques par comptage

In [38]:
def extract_car(vocabList:list,index:list)->list:
    # Créer une liste vide de taille meme taille que vocabList
    caracteristic_vect = [0] * len(vocabList)
    #Utiliser l'index créé dans l'étape d'indexation pour remplir le vecteur des caractéristique la taille du vecteur est eagle au nombre de mots du 
    # vocabulaire et qui sera initialiser à 0 et à chaque apparition du i eme mot faire la mise a jour du vecteur caracteristiques
    #Parcourir chaque element dans index 
    for i in range(len(index)):
            #incrementer la case[index] de 1 dans caracteristic_vect
            caracteristic_vect[index[i]]+=1
            
    return caracteristic_vect

In [39]:
def extract_car_emails(vocabList:list,indexes:list)->list:
    allEmailsCarac=list()
    for i in tqdm(range(len(indexes))):
        allEmailsCarac.append(extract_car(vocabList,indexes[i]))
    return allEmailsCarac


In [40]:
vocabulary=get_vocabulary('vocab.txt')
vocabulary
len(vocabulary)

31550

In [ ]:
indexes=generateAllIndexes(vocabulary,X)

In [46]:
features=extract_car_emails(vocabulary,indexes)
len(features[0])


100%|██████████| 9349/9349 [00:03<00:00, 2790.68it/s]


31550

In [53]:
featuresDF=pd.DataFrame(features)
df=pd.concat([pd.DataFrame(data['label_num']),pd.DataFrame(data['label']),featuresDF],axis=1)
df.head()

,label_num,label,0,1,2,3,4,5,6,7,...,31540,31541,31542,31543,31544,31545,31546,31547,31548,31549
0,0,easy_ham,14,11,14,43,17,1,2,10,...,0,0,0,0,0,0,0,0,0,0
1,0,easy_ham,10,0,0,45,13,1,1,9,...,0,0,0,0,0,0,0,0,0,0
2,0,easy_ham,4,0,0,15,5,1,1,3,...,0,0,0,0,0,0,0,0,0,0
3,0,easy_ham,10,0,0,39,12,1,2,10,...,0,0,0,0,0,0,0,0,0,0
4,0,easy_ham,10,0,0,31,13,1,1,6,...,0,0,0,0,0,0,0,0,0,0


In [55]:
df.to_csv('dataset.csv',index=False)